Urban Data Science & Smart Cities <br>
URSP688Y Spring 2026<br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

# Demo 5 - More Tables & Debugging

- More Tables
    - Drop duplicate rows
    - Count rows withing groups
    - Concatenate tables
    - Join columns from another table
- Errors and Debugging

## Data Wrangling with Tables

In [5]:
# Import Pandas package for working with tables
import pandas as pd

In [6]:
# Load raw eviction data
eviction_cases_df = pd.read_csv('District_Court_of_Maryland_Eviction_Case_Data_MG_PG.csv')

In [7]:
# Investivate the dataframe
eviction_cases_df.head()

,Unnamed: 0,Event Date,Event Type,Event Comment,County,Location,Tenant City,Tenant State,Tenant ZIP Code,Case Type,Case Number,Evicted Date,Event Year,Eviction Year
0,535,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20910.0,Failure to Pay Rent,D-061-LT-22-004107,12/08/2022,2023,2022.0
1,536,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20910.0,Failure to Pay Rent,D-061-LT-22-000755,12/08/2022,2023,2022.0
2,545,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Bethesda,MD,20814.0,Failure to Pay Rent,D-061-LT-22-000816,12/07/2022,2023,2022.0
3,555,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20902.0,Failure to Pay Rent,D-061-LT-22-006362,12/08/2022,2023,2022.0
4,568,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Rockville,MD,20850.0,Failure to Pay Rent,D-061-LT-22-004268,12/07/2022,2023,2022.0


In [8]:
# Data types for each column
eviction_cases_df.dtypes

Unnamed: 0           int64
Event Date          object
Event Type          object
Event Comment       object
County              object
Location            object
Tenant City         object
Tenant State        object
Tenant ZIP Code    float64
Case Type           object
Case Number         object
Evicted Date        object
Event Year           int64
Eviction Year      float64
dtype: object

In [9]:
# Convert data in date columns to datetime objects so they can be sorted properly

def convert_column_to_datetime(df, column):
    """
    Converts a column in a dataframe to datetime objects

    Args:
        df (Pandas DataFrame): input dataframe
        column (string): column in input dataframe to be converted to datetime

    Returns:
        Pandas DataFrame: copy of input dataframe with the converted datetime column
    """
    df = df.copy()
    df[column] = pd.to_datetime(df[column])
    return df

for date_column in ['Event Date', 'Evicted Date']:
    eviction_cases_df = convert_column_to_datetime(eviction_cases_df, date_column)

In [10]:
# Confirm data type changes for date columns
eviction_cases_df.dtypes

Unnamed: 0                  int64
Event Date         datetime64[ns]
Event Type                 object
Event Comment              object
County                     object
Location                   object
Tenant City                object
Tenant State               object
Tenant ZIP Code           float64
Case Type                  object
Case Number                object
Evicted Date       datetime64[ns]
Event Year                  int64
Eviction Year             float64
dtype: object

How many unique cases are there?

In [11]:
# Unique events
len(eviction_cases_df)

160482

In [12]:
# .drop_duplicates() method

# unique cases
eviction_cases_dedup_df = eviction_cases_df.drop_duplicates('Case Number')
len(eviction_cases_dedup_df)

99891

How many unique cases per zip code?

In [13]:
# .groupby() method

eviction_cases_dedup_df.groupby('Tenant ZIP Code').size()

Tenant ZIP Code
19802.0    1
20001.0    2
20010.0    1
20011.0    2
20074.0    1
          ..
28014.0    2
29094.0    1
29747.0    1
29748.0    2
55403.0    1
Length: 147, dtype: int64

In [14]:
# Most recent eviction year for each zip code
eviction_cases_dedup_df.groupby('Tenant ZIP Code')['Event Year'].max()

Tenant ZIP Code
19802.0    2023
20001.0    2024
20010.0    2024
20011.0    2024
20074.0    2024
           ... 
28014.0    2025
29094.0    2025
29747.0    2025
29748.0    2025
55403.0    2023
Name: Event Year, Length: 147, dtype: int64

In [15]:
# Multiple summaries at once: most recent eviction year, count of events, and count of cases per zip code
eviction_cases_df.groupby('Tenant ZIP Code').agg({
    'Event Year': 'max',
    'Event Date': 'count',
    'Case Number': 'nunique'
})

,Event Year,Event Date,Case Number
Tenant ZIP Code,,,
19802.0,2023,2,1
20001.0,2024,4,2
20002.0,2023,1,1
20010.0,2025,3,2
20011.0,2025,4,2
...,...,...,...
29747.0,2025,1,1
29748.0,2025,4,3
29872.0,2025,2,1


Which zip codes have the most unique cases *per person*?

Let's join data from [CensusReporter](https://censusreporter.org/).

### Combining/Merging/Joining Tables

Combining information from multiple tables into a single table is one of the most useful data wrangling operations.

There are lots of different ways to join tables, but two basic types are:

1. Joining column with a shared key, which outputs a table that is wider than either input.
2. Concatenating rows with shared column names, which outputs a table that is longer than either input.

#### Joining columns based on a key

![joining columns with a shared key](https://rforhr.com/horizontal_join.png)


#### Concatenating rows with the same column names
![joining rows with shared column names](https://rforhr.com/vertical_join.png)

In [16]:
# Load census reporter data, ignoring the row with data for the whole county (first row under the header)
acs_mg_df = pd.read_csv('acs2024_5yr_B01003_mg.csv')
acs_pg_df = pd.read_csv('acs2024_5yr_B01003_pg.csv')

In [17]:
# Can we write a function that loads a census reporter csv and skips the row containing countywide data?

def load_census_reporter_population_csv(path):
    """
    Description
    
    Inputs

    Output
    """
    df = pd.read_csv(path) # is there an argument to remove first row?
    # take all rows except the first 3
    df = df.iloc[1:]
    # reset the index to start at 0
    df = df.reset_index(drop=True)
    # rename columns for readability
    df = df.rename(columns={
        'name': 'zip',
        'B01003001':'population'
    })
    # drop unnecessary columns
    df = df.drop(columns=['geoid','B01003001, Error'])
    # store zip codes by strings
    df['zip'] = df['zip'].astype('string')
    df['population'] = df['population'].astype('int64')
    return df

acs_mg_df = load_census_reporter_population_csv('acs2024_5yr_B01003_mg.csv')
acs_pg_df = load_census_reporter_population_csv('acs2024_5yr_B01003_pg.csv')


In [18]:
# Combine into a single dataframe

# pd.concat() function

acs_df = pd.concat([acs_mg_df, acs_pg_df]).reset_index(drop=True)

In [19]:
acs_df

,zip,population
0,20705,28713
1,20707,37278
2,20777,3221
3,20812,276
4,20814,32795
...,...,...
88,20784,33831
89,20785,40066
90,20903,24815
91,20904,58646


In [20]:
# Make sure zip codes are formatted without decimals and stored as strings 
eviction_cases_df['Tenant ZIP Code'] = eviction_cases_df['Tenant ZIP Code'].astype('Int64').astype('string')

In [21]:
# Join zip population into eviction cases
eviction_cases_df = eviction_cases_df.merge(acs_df, left_on='Tenant ZIP Code', right_on='zip')

In [22]:
# Count unique cases per zip code and retain population
cases_by_zip_df = eviction_cases_df.groupby('Tenant ZIP Code').agg({
    'Case Number':'nunique',
    'population': 'first',
})


In [23]:
# Calculate cases per population
cases_by_zip_df['Case Number'] / cases_by_zip_df['population']

Tenant ZIP Code
20601    0.000148
20607    0.003273
20608    0.007215
20613    0.009039
20623    0.001087
           ...   
20905    0.008893
20906    0.038289
20910    0.074027
20912    0.051458
21771    0.000030
Length: 75, dtype: float64

## Errors and debugging

Errors are frustrating and inevitable. Even professional programmers spend much of their time debugging.

Luckily, there are good tools and techniques for making debugging a little easier.

Despite these, you will probably nearly tear your hair out with some frequency, especially as a beginner. It will get better with time.

There are two types of errors in programming: logic and syntax. They both result in your program not achieving its goal, but the first may not be as easily detectable because the code may still run.

### Logic errors
These are issues with how you have approached or executed your problem. If your code runs but produces nonsensical results, there is probably a logic error. However, your erroneous code might also produce logical but *wrong* results; you might never notice until the problem has rippled downstream. It's best to address this proactively by planning your code well so it's less likely to be illogical, and writing readable code that can be easily reviewed.

Here's a logic error. Can you find it? (Hint: the issue is syntactical, but it's still a logic error because the code works without throwing an error.)

In [24]:
def check_adult(age, cutoff=18):
    if age < cutoff:
        adult = False
    else:
        adult = True
    return adult

check_adult(20, cutoff=21)

False

### Syntax errors
These are more obvious because your code will simply fail. There are lots of tools for figuring out where and why.

Error messages are usually the starting place for debugging a syntax error.

In [25]:
def check_adult(age, cutoff=18):
    """
    Check whether an age is legally adult

    Inputs:
    age (int, float): age you want to test
    cutoff (int, float): Optional — Threshold for adulthood

    Return (bool): Test of whether input age is adult
    """
    if age < cutoff:
        adult = False
    else:
        adult = True
    return adult

check_adult('20')

TypeError: '<' not supported between instances of 'str' and 'int'

### Debugging
We can also use an "interactive debugger" to help diagnose our problem by stepping through the code one line at a time.

The debugger allows you to set "breakpoints" where the code will stop running temporarily, a table that shows the values of variables at that time, and buttons to step through the code.

In [26]:
def check_adult(age, cutoff=18):
    if age < cutoff:
        adult = False
    else:
        adult = True
    return adult

check_adult(17)

False